## Load MNIST on Python 3.x

In [2]:
import pickle
import gzip
import numpy as np

In [3]:
filename = 'mnist.pkl.gz'
f = gzip.open(filename, 'rb')
training_data, validation_data, testing_data = pickle.load(f, encoding='latin1')

train_data=np.array(training_data[0])
train_target=np.array(training_data[1])
valid_data=np.array(validation_data[0])
valid_target=np.array(validation_data[1])
test_data=np.array(testing_data[0])
test_target=np.array(testing_data[1])

print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

print(train_target.shape)
print(valid_target.shape)
print(test_target.shape)
f.close()

(50000, 784)
(10000, 784)
(10000, 784)
(50000,)
(10000,)
(10000,)


## Load USPS on Python 3.x

In [4]:
from PIL import Image
import os


In [5]:
USPSMat  = []
USPSTar  = []
curPath  = 'USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)
print(np.array(USPSMat).shape)
print(np.array(USPSTar).shape)
USPS_data=np.array(USPSMat)
USPS_Target=np.array(USPSTar)

(19999, 784)
(19999,)


In [6]:
#Softmax function

def softmax(x):
    #print(x.shape)
    x -= np.max(x)
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [7]:
#One Hot Encoding
import scipy.sparse
def oneHotEncoding(Y):
    m = Y.shape[0]

    oneHot = scipy.sparse.csr_matrix((np.ones(m), (Y, np.array(range(m)))))
    oneHot = np.array(oneHot.todense()).T
    return oneHot
   

In [8]:
oneHotTestTarget=oneHotEncoding(test_target)
oneHotValidTarget=oneHotEncoding(valid_target)
oneHotTrainTarget=oneHotEncoding(train_target)
print(oneHotTrainTarget.shape)
print(oneHotValidTarget.shape)
print(oneHotTestTarget.shape)
print(train_data.shape[0])

(50000, 10)
(10000, 10)
(10000, 10)
50000


In [9]:

def getGradientandLoss(w,x,y,lamda,bias):
    m = x.shape[0] # number of training examples
    y_mat = oneHotEncoding(y) #integer class coding into a one-hot representation
    z = np.dot(x,w) #Then we compute raw class scores given our input and current weights
    
    #print(z[10000])
    
    prob = softmax(z+bias) #perform a softmax to get their probabilities
    
    #print(prob.shape)
    loss = (-1 / m) * np.sum(y_mat * np.log(prob)) + (lamda/2)*np.sum(w*w) #We then find the loss of the probabilities
    grad = (-1 / m) * np.dot(x.T,(y_mat - prob)) + lamda*w #compute the gradient for that loss
    return loss,grad

In [10]:
def getProbsAndPreds(data):
    probs = softmax(np.dot(data,w))
    preds = np.argmax(probs,axis=1)
    return probs,preds

print(train_data.shape[1])
print(len(np.unique(train_target)))

784
10


In [11]:
#Mini-batch stochastic gradient descent
batch_size=100
no_of_batches=500
w = np.zeros([train_data.shape[1],len(np.unique(train_target))])
b = np.zeros([batch_size,len(np.unique(train_target))])
train_data_batched=np.split(train_data, no_of_batches)
train_target_batched=np.split(train_target, no_of_batches)
lamda = 1#10
epochs = 300#
learningRate = 1e-6
losses = []
for i in range(0,epochs):
    for j in range(0,no_of_batches):
        train_data_b=train_data_batched[j]
        train_target_b=train_target_batched[j]
        loss,grad = getGradientandLoss(w,train_data_b,train_target_b,lamda,b)
        losses.append(loss)
        w = w - (learningRate * grad)
    
print(loss)

4.520448063638319


In [17]:
from sklearn.metrics import confusion_matrix
def getAccuracy(data,target):
    prob,prede = getProbsAndPreds(data)
    
    
    accuracy = sum(prede == target)/(float(len(target)))
    return prede,accuracy

In [40]:
softmaxMNISTPred,softmaxMNISTAcc = getAccuracy(test_data,test_target)
softmaxUSPSPred,softmaxUSPSAcc = getAccuracy(USPS_data,USPS_Target)


In [26]:
# Neural Network Implementation


In [27]:
from sklearn.metrics import accuracy_score
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, TensorBoard

Using TensorFlow backend.


In [28]:
def genMNISTModel(RawData):
    input_size = RawData.shape[1]
    drop_out = 0.2
    first_dense_layer_nodes  = 128 
    second_dense_layer_nodes = 64
    
    third_dense_layer_nodes = 10


    model = Sequential()
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('relu'))
    model.add(Dropout(drop_out))



    model.add(Dense(second_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('relu'))

    model.add(Dropout(drop_out))

    model.add(Dense(third_dense_layer_nodes))
    model.add(Activation('softmax'))


    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()
    return model
MNISTmodel=genMNISTModel(train_data)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
__________

In [29]:
from keras.utils import np_utils
def runMNISTModel(train_data,train_target,model):
    
    validation_data_split = 0.2 # This is the validation data split ratio from the training set .This means last 20% of
# data will be used as the validation set.
    num_epochs = 50 # epochs are the number of datasets given to the model to learn.Changing to 1000 for 3 dense layers
# as accuracy is reaching 0.99 in the first 200 epochs only   
    model_batch_size = 100 # No of training examples used per iteration
    tb_batch_size = 32 # batch_size used in callback functions to get the statistics of the model that is getting trained
    early_patience = 25 # no of epochs with no improvement after which training is stopped

# EarlyStopping is used to stop the training when a monitored value stops improving.
    tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
    earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')

    TrainingData = train_data
    TrainingTarget = train_target
    TargetVector = np_utils.to_categorical(TrainingTarget,10)
    #print(TrainingData)
    history = model.fit(TrainingData
                    , TargetVector
                    , validation_data=(valid_data,np_utils.to_categorical(valid_target,10))
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb]
                   )
    return history

history=runMNISTModel(train_data,train_target,MNISTmodel)


Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 23s 470us/step - loss: 0.4591 - acc: 0.8620 - val_loss: 0.1716 - val_acc: 0.9497
Epoch 2/50
50000/50000 [==============================] - 21s 415us/step - loss: 0.1999 - acc: 0.9401 - val_loss: 0.1209 - val_acc: 0.9648
Epoch 3/50
50000/50000 [==============================] - 24s 485us/step - loss: 0.1521 - acc: 0.9548 - val_loss: 0.0992 - val_acc: 0.9707
Epoch 4/50
50000/50000 [==============================] - 28s 557us/step - loss: 0.1233 - acc: 0.9625 - val_loss: 0.0903 - val_acc: 0.9724
Epoch 5/50
50000/50000 [==============================] - 21s 416us/step - loss: 0.1073 - acc: 0.9676 - val_loss: 0.0793 - val_acc: 0.9760
Epoch 6/50
50000/50000 [==============================] - 23s 462us/step - loss: 0.0933 - acc: 0.9710 - val_loss: 0.0872 - val_acc: 0.9735
Epoch 7/50
50000/50000 [==============================] - 23s 464us/step - loss: 0.0889 - acc: 0.9727 - val_loss: 0.

In [37]:
mnistNNPredictions = np.argmax(MNISTmodel.predict(test_data),axis=1)
uspsNNPredictions = np.argmax(MNISTmodel.predict(USPS_data),axis=1)


In [39]:
# SVM CLassifier Implementation


[7 6 7 ... 7 7 7]


In [41]:
from sklearn import datasets, svm, metrics

classifier = svm.SVC(C=200,kernel='linear',gamma=0.05)
classifier.fit(train_data,train_target)

SVC(C=200, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.05, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
mnist_pred_svm = classifier.predict(test_data)
usps_pred_svm = classifier.predict(USPS_data)

In [44]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier


randomforestClassify = RandomForestClassifier(n_jobs=-1, n_estimators=10)

In [45]:
randomforestClassify.fit(train_data,train_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [49]:
mnist_pred_forest = randomforestClassify.predict(test_data)
usps_pred_forest = randomforestClassify.predict(USPS_data)
